## 4514 Lab 5 - Ex 3 (Suggested Solutions)

### Dependencies

In [1]:
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

### Sample Data

In [2]:
observations = torch.tensor([61.5, 67.0, 63.6, 62.4, 66.8])
print(f'The mean of the observations is: {torch.mean(observations)}.')

The mean of the observations is: 64.25999450683594.


### Model

In [3]:
def model(observations):
    weight_prior = dist.Uniform(60.0, 70.0)   
    weight = pyro.sample("weight", weight_prior)
    
    my_dist = dist.Normal(weight, 1.5)
    
    for i, obs in enumerate(observations):
        measurement = pyro.sample(f'obs_{i}', my_dist, obs=obs)

### Guide

In [4]:
def guide(observations):

    # Our intial guesses for the mean (65.0) and standard deviation (1.5)
    mean_parameter = pyro.param("mu", torch.tensor(65.0))
    std_parameter = pyro.param("sigma", torch.tensor(1.5), constraint=constraints.positive)
    
    # We choose a normal distribution to define the weight distribution
    weight_distribution = dist.Normal(mean_parameter, std_parameter)    
    weight = pyro.sample("weight", weight_distribution)

### Inference

In [5]:
# Clear all the parameter store
pyro.clear_param_store()

# Setup the inference algorithm
svi = SVI(model=model,
          guide=guide,
          optim=Adam({"lr": 0.0005}),
          loss=Trace_ELBO())

# Do optimization steps 
n_steps = 5000
for i in range(n_steps):
    loss = svi.step(observations)
    if (i % 100 == 0):
        print(f'iter: {i}, loss: {round(loss,2)}', end="\r")

In [6]:
print("Based on the data and our prior belief, the fairness " +
      "of the body scale is %.3f +- %.3f" % (pyro.param("mu"), pyro.param("sigma")))

Based on the data and our prior belief, the fairness of the body scale is 64.352 +- 0.763
